In [1]:
import pandas as pd
import numpy as np

#Strain Shift Constant
D = 2.87022  # Given value of D in GHz

input_excel_path = "scidavis_table.xlsx"
output_excel_path = "scidavis_table_with_results.xlsx"

data = pd.DataFrame({
    "X": [1.6, 1.8, 2.0, 2.2, 2.4, 2.6],
    "f_minus": [2.86621, 2.86583, 2.86526, 2.86507, 2.86468, 2.8643],
    "f_plus": [2.87629, 2.87705, 2.87838, 2.87933, 2.88047, 2.8818]
})

# Function to compute \beta^2
def compute_beta_squared(row, D):
    f_minus = row['f_minus']
    f_plus = row['f_plus']
    beta_squared = (1/3) * (f_minus**2 + f_plus**2 - f_minus*f_plus - D**2)
    return beta_squared

# Function to compute \Delta
def compute_delta(row, beta_squared, D, E):
    f_minus = row['f_minus']
    f_plus = row['f_plus']
    delta = (1 / (27 * beta_squared)) * (
        7 * D**3
        - 2 * (f_plus + f_minus) * (2 * f_plus - f_minus) * (2 * f_minus - f_plus)
        - 3 * D * (f_minus**2 + f_plus**2 - f_minus*f_plus)
    )
    return delta

# Function to compute cos(2θ) and θ
def compute_cos_theta_and_theta(row, D):
    delta = row['delta']
    cos_2theta = delta / D  # Compute cos(2θ)
    # Ensure the value is in the range [-1, 1] to avoid invalid results
    #cos_2theta = max(min(cos_2theta, 1), -1)
    theta = np.arccos(np.sqrt((1 + cos_2theta) / 2)) * 180 / np.pi  # Compute θ in degrees
    return pd.Series({'cos_2theta': cos_2theta, 'theta': theta})

# Compute beta^2
data['beta_squared'] = data.apply(compute_beta_squared, axis=1, D=D)

# Compute \Delta
data['delta'] = data.apply(lambda row: compute_delta(row, row['beta_squared'], D, E), axis=1)

# Compute cos(2θ) and θ
cos_theta_and_theta = data.apply(compute_cos_theta_and_theta, axis=1, D=D)
data['cos_2theta'] = cos_theta_and_theta['cos_2theta']
data['theta'] = cos_theta_and_theta['theta']

data.to_excel(output_excel_path, index=False, sheet_name="Results")

print(f"Results saved to {output_excel_path}")


Results saved to scidavis_table_with_results.xlsx
